### Постановка задачи
Рассмотрим несколько моделей линейной регрессии, чтобы выяснить более оптимальную для первых 20 зданий.

Данные:
* http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz
* http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz
    
Соревнование: https://www.kaggle.com/c/ashrae-energy-prediction/

© ITtensive, 2020

In [1]:
import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet, BayesianRidge


def reduce_mem_usage(df: pd.DataFrame):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if str(col_type)[:5] == 'float':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.finfo('f2').min and c_max < np.finfo('f2').max:
                df[col] = df[col].astype(np.float16)
            elif c_min > np.finfo('f4').min and c_max < np.finfo('f4').max:
                df[col] = df[col].astype(np.float32)
            else:
                df[col] = df[col].astype(np.float64)
        elif str(col_type)[:3] == 'int':
            c_min = df[col].min()
            c_max = df[col].max()
            if c_min > np.iinfo("i1").min and c_max < np.iinfo("i1").max:
                df[col] = df[col].astype(np.int8)
            elif c_min > np.iinfo("i2").min and c_max < np.iinfo("i2").max:
                df[col] = df[col].astype(np.int16)
            elif c_min > np.iinfo("i4").min and c_max < np.iinfo("i4").max:
                df[col] = df[col].astype(np.int32)
            elif c_min > np.iinfo("i8").min and c_max < np.iinfo("i8").max:
                df[col] = df[col].astype(np.int64)
        elif col == 'timestamp':
            df[col] = pd.to_datetime(df[col])
        elif str(col_type)[:8] != 'datetime':
            df[col] = df[col].astype('category')
    
    end_mem = df.memory_usage().sum() / 1024**2
    print(
        'Потребление памяти меньше на ',
        round(start_mem - end_mem, 2),
        ' Мб (-',
        round(100 * (start_mem - end_mem) / start_mem, 1),
        '%)',
        sep=''
    )
    return df

### Загрузка данных, отсечение 20 зданий, объединение и оптимизация

In [2]:
buildings = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/building_metadata.csv.gz")
weather = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/weather_train.csv.gz")
energy = pd.read_csv("http://video.ittensive.com/machine-learning/ashrae/train.0.csv.gz")

energy = energy[energy['building_id'] < 20]

energy = pd.merge(left=energy, right=buildings, how='left', left_on='building_id', right_on='building_id')
energy = pd.merge(
    left=energy.set_index(['timestamp', 'site_id']), 
    right=weather.set_index(['timestamp', 'site_id']),
    how='left', left_index=True, right_index=True
)
energy.reset_index(inplace=True)
energy = energy.drop(columns=['meter', 'site_id', 'year_built', 'square_feet', 'floor_count'], axis='columns')

del buildings
del weather
energy = reduce_mem_usage(energy)
energy.info()

Потребление памяти меньше на 10.39 Мб (-70.5%)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175680 entries, 0 to 175679
Data columns (total 11 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   timestamp           175680 non-null  datetime64[ns]
 1   building_id         175680 non-null  int8          
 2   meter_reading       175680 non-null  float16       
 3   primary_use         175680 non-null  category      
 4   air_temperature     175620 non-null  float16       
 5   cloud_coverage      99080 non-null   float16       
 6   dew_temperature     175620 non-null  float16       
 7   precip_depth_1_hr   175660 non-null  float16       
 8   sea_level_pressure  173980 non-null  float16       
 9   wind_direction      170680 non-null  float16       
 10  wind_speed          175680 non-null  float16       
dtypes: category(1), datetime64[ns](1), float16(8), int8(1)
memory usage: 4.4 MB


### Обогащение данных: час, дни недели, праздники, логарифм

In [3]:
energy['hour'] = energy['timestamp'].dt.hour.astype('int8')
energy['weekday'] = energy['timestamp'].dt.weekday.astype('int8')
for weekday in range(0, 7):
    energy['is_wday' + str(weekday)] = energy['weekday'].isin([weekday]).astype('int8')
energy['date'] = pd.to_datetime(energy['timestamp'].dt.date)
dates_range = pd.date_range(start='2015-12-31', end='2017-01-01')
us_holidays = calendar().holidays(start=dates_range.min(), end=dates_range.max())
energy['is_holiday'] = energy['date'].isin(us_holidays).astype('int8')
energy['meter_reading_log'] = np.log(energy['meter_reading'] + 1)

### Разделение данных

In [7]:
energy_train, energy_test = train_test_split(energy[(energy['meter_reading'] > 0)], test_size=0.2)
energy_train.head()

,timestamp,building_id,meter_reading,primary_use,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,...,is_wday0,is_wday1,is_wday2,is_wday3,is_wday4,is_wday5,is_wday6,date,is_holiday,meter_reading_log
148831,2016-11-06 01:00:00,11,482.000,Education,20.593750,2.0,14.398438,0.0,1021.5,10.0,...,0,0,0,0,0,0,1,2016-11-06,0,6.179688
137013,2016-10-12 10:00:00,13,465.250,Education,21.703125,NaN,20.000000,0.0,1018.0,350.0,...,0,0,1,0,0,0,0,2016-10-12,0,6.144531
120275,2016-09-07 13:00:00,15,412.500,Office,26.703125,0.0,21.093750,0.0,1021.5,50.0,...,0,0,1,0,0,0,0,2016-09-07,0,6.023438
156046,2016-11-21 02:00:00,6,137.625,Lodging/residential,12.796875,0.0,0.600098,0.0,1021.0,310.0,...,1,0,0,0,0,0,0,2016-11-21,0,4.933594
102811,2016-08-02 04:00:00,11,479.250,Education,26.703125,2.0,23.296875,0.0,1020.5,120.0,...,0,1,0,0,0,0,0,2016-08-02,0,6.175781


### Линейная регрессия: по часам

In [8]:
from sklearn.metrics import r2_score

hours = range(0, 24)
buildings = range(0, energy_train['building_id'].max() + 1)
lr_columns = ['meter_reading_log', 'hour', 'building_id', 'is_holiday']
lr_columns.extend(['is_wday' + str(wday) for wday in range(0, 7)])
lr_models = {
    'LinearRegression': LinearRegression,
    'Lasso-0.01': Lasso,
    'Lasso-0.1': Lasso,
    'Lasso-1.0': Lasso,
    'Ridge-0.01': Ridge,
    'Ridge-0.1': Ridge,
    'Ridge-1.0': Ridge,
    'ElasticNet-1-1': ElasticNet,
    'ElasticNet-0.1-1': ElasticNet,
    'ElasticNet-1-0.1': ElasticNet,
    'ElasticNet-0.1-0.1': ElasticNet,
    'BayesianRidge': BayesianRidge
}
energy_train_lr = pd.DataFrame(energy_train, columns=lr_columns)

Линейная регрессия
\begin{equation}
z = Ax + By + C, |z-z_0|^2 \rightarrow min
\end{equation}

Лассо + LARS Лассо
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + a(|A|+|B|) \rightarrow min
\end{equation}

Гребневая регрессия
\begin{equation}
|z-z_0|^2 + a(A^2 + B^2) \rightarrow min
\end{equation}

ElasticNet: Лассо + Гребневая регрессия
\begin{equation}
\frac{1}{2n}|z-z_0|^2 + \alpha p|A^2+B^2| + (\alpha - p)(|A|+|B|)/2 \rightarrow min
\end{equation}

In [19]:
lr_models_scores = {}
for _ in lr_models:
    lr_model = lr_models[_]
    energy_lr_scores = [[]]*len(buildings)
    for building in buildings:
        energy_lr_scores[building] = [0]*len(hours)
        energy_train_b = energy_train_lr[energy_train_lr['building_id'] == building]
        for hour in hours:
            energy_train_bh = energy_train_b[energy_train_b['hour'] == hour]
            y = energy_train_bh['meter_reading_log']
            x = energy_train_bh.drop(labels=['meter_reading_log', 'hour', 'building_id'], axis=1)
            if _ in ['Ridge-0.1', 'Lasso-0.1']:
                model = lr_model(alpha=.1, fit_intercept=False).fit(x, y)
            elif _ in ['Ridge-0.01', 'Lasso-0.01']:
                model = lr_model(alpha=.01, fit_intercept=False).fit(x, y)
            elif _ == 'ElasticNet-1-1':
                model = lr_model(alpha=1, l1_ratio=1, fit_intercept=False).fit(x, y)
            elif _ == 'ElasticNet-1-0.1':
                model = lr_model(alpha=1, l1_ratio=.1, fit_intercept=False).fit(x, y)
            elif _ == 'ElasticNet-0.1-1':
                model = lr_model(alpha=.1, l1_ratio=1, fit_intercept=False).fit(x, y)
            elif _ == 'ElasticNet-0.1-0.1':
                model = lr_model(alpha=.1, l1_ratio=.05, fit_intercept=False).fit(x, y)
            else:
                model = lr_model(fit_intercept=False).fit(x, y)
            energy_lr_scores[building][hour] = r2_score(y, model.predict(x))
    lr_models_scores[_] = np.mean(energy_lr_scores)

In [20]:
lr_models_scores

{'LinearRegression': 0.13168010519118914,
 'Lasso-0.01': -0.19122762842848176,
 'Lasso-0.1': -31.167637948440845,
 'Lasso-1.0': -2434.1731246611316,
 'Ridge-0.01': 0.1312697839560794,
 'Ridge-0.1': 0.09100067585954111,
 'Ridge-1.0': -3.619062066977636,
 'ElasticNet-1-1': -2434.1731246611316,
 'ElasticNet-0.1-1': -31.167637948440845,
 'ElasticNet-1-0.1': -2017.4958981604113,
 'ElasticNet-0.1-0.1': -423.2530209353185,
 'BayesianRidge': 0.13167029470206285}

### Проверим модели: LinearRegression, Lasso, BayesianRidge

In [21]:
energy_lr = [[]]*len(buildings)
energy_lasso = [[]]*len(buildings)
energy_br = [[]]*len(buildings)
for building in buildings:    
    energy_train_b = energy_train_lr[energy_train_lr['building_id']==building]
    energy_lr[building] = [[]]*len(hours)
    energy_lasso[building] = [[]]*len(hours)
    energy_br[building] = [[]]*len(hours)
    for hour in hours:
        energy_train_bh = pd.DataFrame(energy_train_b[energy_train_b['hour']==hour])
        y = energy_train_bh['meter_reading_log']
        if len(y) > 0:
            x = energy_train_bh.drop(labels=['meter_reading_log', 'hour', 'building_id'], axis=1)
            model = LinearRegression(fit_intercept=False).fit(x, y)
            energy_lr[building][hour] = np.append([model.coef_], model.intercept_)
            model = Lasso(fit_intercept=False, alpha=.01).fit(x, y)
            energy_lasso[building][hour] = np.append([model.coef_], model.intercept_)
            model = BayesianRidge(fit_intercept=False).fit(x, y)
            energy_br[building][hour] = np.append([model.coef_], model.intercept_)
print (energy_lr[0][0])
print (energy_lasso[0][0])
print (energy_br[0][0])

[-0.1016428   5.42939937  5.52428668  5.49884259  5.43734059  5.47444996
  5.44450431  5.46947338  0.        ]
[-0.          5.34453125  5.44602582  5.43217593  5.35810547  5.39521484
  5.38243534  5.40280671  0.        ]
[-0.10117087  5.42895122  5.52385473  5.49847632  5.43691348  5.47402007
  5.44416666  5.46910906  0.        ]
